In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from lib.pipeline import Pipeline
import torch
GPU = 0
pipeline = Pipeline(
    model='lm-gearnet',
    dataset='atpbind3d-minimal',
    gpus=[GPU],
    model_kwargs={
        'gpu': 0,
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'bert_freeze': False,
        'bert_freeze_layer_count': 29,
    }
)

state_dict = torch.load('../lmg_29_4_512_0.58736.pth', map_location=f'cuda:{GPU}')
pipeline.model.load_state_dict(state_dict)


get dataset atpbind3d-minimal
Split num:  [4, 1, 5]
get item 0


ValueError: Expect residue attribute `target` to have shape (350, *), but found torch.Size([566, 1])

In [ ]:
# single pass의 경우 prediction과 loss가 어떻게 나오는지 확인?
from torchdrug import data, utils
dataloader = data.DataLoader([pipeline.train_set[0]], batch_size=1, shuffle=False)
batch = utils.cuda(next(iter(dataloader)), device=torch.device('cuda:{}'.format(GPU)))
pred, target = pipeline.task.predict_and_target(batch)
pred.shape

get item 0


torch.Size([350, 1])

In [ ]:
target_index = [i+1 for i, item in enumerate(target['label']) if item.item() == 1]
print(target_index)

[315, 316, 317, 321, 322, 324, 333, 335]


In [ ]:
print(pred[310:330])

tensor([[-0.1920],
        [ 0.1383],
        [-0.2469],
        [ 0.1761],
        [-0.0894],
        [ 0.1021],
        [ 0.0327],
        [-0.0528],
        [-0.0554],
        [ 0.2078],
        [ 0.1084],
        [ 0.2347],
        [-0.0943],
        [ 0.0247],
        [-0.1133],
        [-0.3579],
        [-0.1789],
        [-0.2484],
        [ 0.0170],
        [-0.0496]], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
print(target['label'].float()[310:330])

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0')


In [ ]:
pipeline.calculate_best_mcc_and_threshold()

get item 4


{'best_mcc': 0.0018134073025168724, 'best_threshold': 0.30000000000000027}

In [ ]:
from torch.nn import functional as F
loss = F.binary_cross_entropy_with_logits(pred, target["label"].float(), reduction="none")
loss

tensor([[0.6514],
        [0.7210],
        [0.7564],
        [0.7950],
        [0.8091],
        [0.6705],
        [0.7236],
        [0.7576],
        [0.7934],
        [0.6160],
        [0.6972],
        [0.8441],
        [0.7898],
        [0.7273],
        [0.7017],
        [0.7419],
        [0.7049],
        [0.7446],
        [0.8527],
        [0.6859],
        [0.7753],
        [0.7505],
        [0.6396],
        [0.6227],
        [0.5798],
        [0.7367],
        [0.7874],
        [0.6646],
        [0.6835],
        [0.8143],
        [0.6852],
        [0.6774],
        [0.7420],
        [0.6523],
        [0.7458],
        [0.6750],
        [0.8594],
        [0.7974],
        [0.9411],
        [0.7921],
        [0.7792],
        [0.8310],
        [0.8617],
        [0.6746],
        [0.7339],
        [0.7546],
        [0.8546],
        [0.6732],
        [0.7331],
        [0.6850],
        [0.7856],
        [0.7676],
        [0.7071],
        [0.8031],
        [0.7337],
        [0

In [ ]:
import torch

def generate_random_undersampling_mask(target):
    # Count the number of positive samples
    n_positives = torch.sum(target > 0)

    # Compute the probability to get equal number of negative samples
    negative_prob = 0.1

    print(f'negative_prob: {negative_prob}')
    torch.random.manual_seed(4)

    # Generate random numbers between 0 and 1
    random_tensor = torch.rand(target.size())

    # Create a mask where the values are 1 if the target is positive or if the random number is lower than the computed probability
    mask = ((target > 0) + (random_tensor < negative_prob))

    return mask

generate_random_undersampling_mask(torch.tensor([0, 0, 0, 0, 0, 1, 0, 1, 0, 0]))


negative_prob: 0.25


tensor([False, False,  True,  True,  True,  True, False,  True, False, False])

In [ ]:
batch['graph'].mask = generate_random_undersampling_mask(batch['graph'].target.to('cpu')).to(f'cuda:{GPU}')

negative_prob: 0.023391813039779663


In [ ]:
from torchdrug.layers import functional
loss_all = functional.masked_mean(loss, torch.ones_like(loss).bool(), dim=0)
loss_all

tensor([0.7275], device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
loss_rus = functional.masked_mean(loss, batch['graph'].mask, dim=0)
loss_rus

tensor([0.6944], device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
all_loss = torch.tensor(0, dtype=torch.float32, device=f'cuda:{GPU}').view(1)

In [ ]:
pipeline.test_set[0]

get item 5


{'graph': Protein(num_atom=1400, num_bond=2796, num_residue=350)}